# Structured Outputs with llm_generate using the vLLM Provider

<a target="_blank" href="https://colab.research.google.com/github/everettVT/daft-structured-outputs/blob/main/workload/llm_generate_vllm_provider_examples.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Dont forget to follow installation and setup instructions in the readme.

WARNING: vLLM has experimental support for macOS with Apple Silicon. For now, users must build from source to natively run on macOS.

Colab or Intel/AMD Recommended for CPU.


In [2]:
!pip install daft vllm pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0

#### Authenticate with HuggingFace for Access to Gemma-3 Series models

In [1]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
The token `Anyscale Ray Serve LLM` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

In [8]:
import daft
from daft import col
from daft.functions import llm_generate, format
from vllm.sampling_params import GuidedDecodingParams

MAX_TOKENS = 100
TEMPERATURE = 0.0
NUM_GPUS = 1

## Guided Choice

In [5]:
df_choice = daft.from_pydict({
    "text": [
        "I'm not a fan of slow data pipelines",
        "Daft Dataframes are wicked fast!",
    ]
})

df_choice = df_choice.with_column("sentiment", llm_generate(
    "Classify this sentiment: " + df_choice["text"],
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding= GuidedDecodingParams(choice=["Positive", "Negative"]),
    max_tokens= MAX_TOKENS,
    temperature= TEMPERATURE,
    num_gpus= NUM_GPUS
)).collect()
df_choice.show()

textUtf8,sentimentUtf8
I'm not a fan of slow data pipelines,Positive
Daft Dataframes are wicked fast!,Negative


## Guided Regex

In [10]:
df_regex = daft.from_pydict({
    "name": [
        "John Doe",
        "Jane Smith",
        "Alice Johnson",
        "Bob Brown",
        "Charlie Davis",
    ],
    "company": [
        "Acme Inc.",
        "Globex Corp.",
        "Initech",
        "Soylent Corp.",
        "Umbrella Corp.",
    ]
})

df_regex = df_regex.with_column("email", llm_generate(
    format("""
    Generate an email address for {} at {}
    End in .com
    """, col("name"), col("company")),
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(regex=r"\w+@\w+\.com\n"),
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    num_gpus=NUM_GPUS
))
df_regex.show()


nameUtf8,companyUtf8,emailUtf8
John Doe,Acme Inc.,GenerateanemailaddressforJohnDoeatAcmeInc@com.com
Jane Smith,Globex Corp.,GenerateanemailaddressforJaneSmithatGlobexCorp@gmail.com
Alice Johnson,Initech,GenerateanemailaddressforAliceJohnsonatInitech@gmail.com
Bob Brown,Soylent Corp.,GenerateanemailaddressforBobBrownatSoylentCorp@gmail.com
Charlie Davis,Umbrella Corp.,GenerateanemailaddressforCharlieDavisatUmpireCorp@gmail.com


## Guided Decoding by Pydantic JSON Schema

In [12]:
from pydantic import BaseModel, Field
from enum import Enum

class CarType(str, Enum):
    sedan = "sedan"
    suv = "SUV"
    truck = "Truck"
    coupe = "Coupe"


class CarDescription(BaseModel):
    make: str = Field(description="The make of the car")
    model: str = Field(description="The model name of the car")
    car_type: CarType = Field(description="The type of vehicle")


df_pydantic = daft.from_pydict({
    "decade": [
        "80's",
        "90's",
        "2000's",
    ]
})

df_pydantic = df_pydantic.with_column("car_description_json", llm_generate(
    format("Generate a car description for the {} decade", col("decade")),
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(json=CarDescription.model_json_schema()),
    max_tokens=MAX_TOKENS,
    temperature=TEMPERATURE,
    num_gpus=NUM_GPUS
))

df_pydantic = df_pydantic.with_column("car_description_validated",
    df_pydantic["car_description_json"].apply(
        lambda x: CarDescription.model_validate_json(x),
        return_dtype= daft.DataType.python()
    )
)
df_pydantic.show()


decadeUtf8,car_description_jsonUtf8,car_description_validatedPython
80's,"{ ""brand"": ""Ford"", ""model"": ""Ford Mustang"", ""car_type"": ""SUV""}",brand='Ford' model='Ford Mustang' car_type=<CarType.suv: 'SUV'>
90's,"{ ""brand"": ""90s"", ""model"": ""90s"", ""car_type"": ""SUV""}",brand='90s' model='90s' car_type=<CarType.suv: 'SUV'>
2000's,"{ ""brand"": ""Toyota"", ""model"": ""Toyota"", ""car_type"": ""SUV""}",brand='Toyota' model='Toyota' car_type=<CarType.suv: 'SUV'>


## Guided Decoding by Grammar

In [13]:
simplified_sql_grammar = """
root ::= select_statement
select_statement ::= "SELECT " column " from " table " where " condition
column ::= "col_1 " | "col_2 "
table ::= "table_1 " | "table_2 "
condition ::= column "= " number
number ::= "1 " | "2 "
limit ::= "LIMIT " number
"""

df_grammar = daft.from_pydict({
    "prompt": [
        "Generate an SQL query to show the 'username' and 'email' from the 'users' table.",
        "Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30.",
        "Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30 and the name is 'John Doe'.",
        "Show me the first 10 rows of the 'users' table.",
    ]
})

df_grammar = df_grammar.with_column("sql_query", llm_generate(
    df_grammar["prompt"],
    model="google/gemma-3-270m",
    provider="vllm",
    guided_decoding=GuidedDecodingParams(grammar=simplified_sql_grammar),
    num_gpus=NUM_GPUS
))
df_grammar.show()

promptUtf8,sql_queryUtf8
Generate an SQL query to show the 'username' and 'email' from the 'users' table.,SELECT col_1 from table_1 where
Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30.,SELECT col_1 from table_2 where col_1
Generate an SQL query to show the 'name' and 'age' from the 'users' table where the age is greater than 30 and the name is 'John Doe'.,SELECT col_1 from table_1 where col_
Show me how to see the first 10 rows of the 'users' table.,SELECT col_1 from table_1 where col_
